# Описание
- Оцениваем расхождение модельных значений коэффициентов и коэффициентов из линии по разным букмекерам
- Входной файл - dwh_pre.csv

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

sys.path.append('./')
sys.path.append('./operator_position')

In [3]:
data_4 = pd.read_csv('/Users/i.tolkachev/freelance/softpro/dwh_pre.csv')

In [4]:
data_4.shape

(3967436, 11)

In [5]:
data_4.head()

,Unnamed: 0,QueueTime,InternalID,StartTime,BookmakerName,LineKindName,IntervalKindName,MarketKindName,SelectionKindName,MarketValue,Price
0,0,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Home,-0.25,3.150
1,1,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Away,-0.25,1.395
2,2,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Home,-0.50,3.640
3,3,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Away,-0.50,1.315
4,4,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Home,0.25,2.090


In [6]:
data_4.InternalID.nunique()

618

In [7]:
def create_odds(m_k_name, s_k_name, m_val, price):
    return dict(
        zip(
            m_k_name + ' ' + s_k_name + ' ' + m_val.astype(str).tolist(),
            price.tolist()
        )
    )

In [8]:
from tqdm import tqdm, tqdm_notebook, tqdm_pandas

tqdm_notebook().pandas()

In [9]:
data_4_odds_dict = (
    data_4
    .groupby(['QueueTime', 'InternalID', 'StartTime', 'BookmakerName', 'LineKindName', 'IntervalKindName'])
    [['MarketKindName', 'SelectionKindName', 'MarketValue', 'Price']]
    .progress_apply(lambda x:
        create_odds(x['MarketKindName'], x['SelectionKindName'], x['MarketValue'], x['Price'])
    )
)

In [10]:
data_4.groupby('InternalID')['MarketKindName'].count().head()

InternalID
00e3c7b3-ad36-447a-82b6-3dd1860ba1a3     4009
010cdc53-3c55-47dc-b99b-eabf9ddc4854      957
011c00aa-e567-4425-aed8-6dab1d08d137    31898
01a826ea-805f-4851-bc39-b1c9711148f2     4095
01c77461-34c9-4f70-9f9a-9a779b5a296c    30856
Name: MarketKindName, dtype: int64

In [11]:
data_4['MarketKindName'].unique()

array(['Handicap', 'Total', 'ThreeWay', 'HomeTeamTotal', 'AwayTeamTotal',
       'BothTeamToScore', 'HomeToWinToNil', 'AwayToWinToNil'],
      dtype=object)

In [12]:
data_4[data_4['MarketKindName'] == 'HomeTeamTotal'].head()

,Unnamed: 0,QueueTime,InternalID,StartTime,BookmakerName,LineKindName,IntervalKindName,MarketKindName,SelectionKindName,MarketValue,Price
130,204,2019-12-01 12:00:20,e89ce591-7ec4-44f1-b30f-b6f7cdacb1e7,2019-12-01 19:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Over,1.5,1.704
131,205,2019-12-01 12:00:20,e89ce591-7ec4-44f1-b30f-b6f7cdacb1e7,2019-12-01 19:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Under,1.5,2.180
163,237,2019-12-01 12:00:20,e8cc72bd-48b8-40ed-a06c-e6d7eb1cfe8d,2019-12-01 14:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Over,1.5,1.813
164,238,2019-12-01 12:00:20,e8cc72bd-48b8-40ed-a06c-e6d7eb1cfe8d,2019-12-01 14:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Under,1.5,2.070
363,546,2019-12-01 12:00:40,e89ce591-7ec4-44f1-b30f-b6f7cdacb1e7,2019-12-01 19:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Over,1.5,1.675


In [14]:
data_4_odds_dict = data_4_odds_dict.reset_index()

In [15]:
data_4_odds_dict.head()

,QueueTime,InternalID,StartTime,BookmakerName,LineKindName,IntervalKindName,0
0,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,"{'Handicap Home -0.25': 3.15, 'Handicap Away -..."
1,2019-12-01 12:00:02,8a143bfb-1700-4d3c-b2d8-480d7f2f8f9d,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,"{'Handicap Home -0.5': 3.91, 'Handicap Away -0..."
2,2019-12-01 12:00:05,91fd064b-5ecc-4e9c-af3f-0a258e5a9394,2019-12-01 16:30:00,bookmaker1,PreMatch,FullTime,"{'Handicap Home -2.0': 2.36, 'Handicap Away -2..."
3,2019-12-01 12:00:08,0aa80a7e-1613-41e3-af78-f1d26382e386,2019-12-01 18:00:00,bookmaker2,PreMatch,FullTime,"{'Total Over 2.25': 1.88, 'Total Under 2.25': ..."
4,2019-12-01 12:00:08,26aab835-071d-40a8-bc53-86baf36037bc,2019-12-01 16:30:00,bookmaker2,PreMatch,FullTime,"{'ThreeWay Home 0.0': 4.8, 'ThreeWay Away 0.0'..."


In [17]:
# get standard column names
data_3 = pd.read_excel('/Users/i.tolkachev/freelance/softpro/20191125.xlsx', sheet_name='Sheet2')

columns_data = [c for c in data_3.columns.tolist() if not ('computed' in c or 'disc' in c)]

ml_columns = ['HOME', 'DRAW', 'AWAY']
total_columns = [col for col in columns_data if 'TO' in col or 'TU' in col]
total_home_columns = [col for col in columns_data if 'T1O' in col or 'T1U' in col]
total_away_columns = [col for col in columns_data if 'T2O' in col or 'T2U' in col]
hdp_columns = [col for col in columns_data if 'HDP' in col]

In [18]:
# map column formats (pinnacle --- others)
map_col = {}
map_col_inv = {}
for c in hdp_columns:
    cur = 'Handicap'
    if c[:4] == 'HOME':
        cur += ' Home'
    else:
        cur += ' Away'
    hdp_val = c.split(' ')[1].replace(',', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c
    
for c in total_columns:
    cur = 'Total'
    if c[1] == 'O':
        cur += ' Over'
    else:
        cur += ' Under'
    hdp_val = c[3:].replace('_', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c
    
for c in total_home_columns:
    cur = 'HomeTeamTotal'
    if c[2] == 'O':
        cur += ' Over'
    else:
        cur += ' Under'
    hdp_val = c[4:].replace('_', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c

for c in total_away_columns:
    cur = 'AwayTeamTotal'
    if c[2] == 'O':
        cur += ' Over'
    else:
        cur += ' Under'
    hdp_val = c[4:].replace('_', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c

for c in ml_columns:
    cur = 'ThreeWay'
    if c == 'HOME':
        cur += ' Home'
    elif c == 'DRAW':
        cur += ' Draw'
    else:
        cur += ' Away'
    cur += f' 0.0'
    map_col[c] = cur
    map_col_inv[cur] = c

In [19]:
def map_col_names(x):
    ans = {}
    for k,v in x.items():
        key = map_col_inv.get(k)
        if key:
            ans[map_col_inv.get(k)] = v
    return ans

In [20]:
data_4_odds_dict['norm_odds'] = data_4_odds_dict[0].progress_apply(map_col_names)

In [21]:
book_odds = data_4_odds_dict['norm_odds'].progress_apply(pd.Series)

In [22]:
book_odds[['QueueTime', 'InternalID', 'StartTime', 'BookmakerName', 'LineKindName']] = (
    data_4_odds_dict[['QueueTime', 'InternalID', 'StartTime', 'BookmakerName', 'LineKindName']]
)

In [23]:
book_odds.head(1)

,"HOME_HDP -0,25","AWAY_HDP -0,25","HOME_HDP -0,5","AWAY_HDP -0,5","HOME_HDP 0,25","AWAY_HDP 0,25","HOME_HDP 1,5","AWAY_HDP 1,5","HOME_HDP 0,75","AWAY_HDP 0,75","HOME_HDP 0,5","AWAY_HDP 0,5","HOME_HDP 0,0","AWAY_HDP 0,0",TO_2_75,TU_2_75,TO_2_0,TU_2_0,TO_3_0,TU_3_0,TO_3_5,TU_3_5,TO_4_5,TU_4_5,TO_2_5,TU_2_5,TO_1_5,TU_1_5,TO_2_25,TU_2_25,HOME,AWAY,DRAW,"HOME_HDP 1,0","AWAY_HDP 1,0","HOME_HDP -2,0","AWAY_HDP -2,0","AWAY_HDP 2,25","HOME_HDP -0,75","AWAY_HDP -0,75","HOME_HDP -1,25","HOME_HDP -1,0","AWAY_HDP -1,0","HOME_HDP -1,5","AWAY_HDP -1,5","HOME_HDP -2,5","AWAY_HDP -2,5",TO_3_25,TU_3_25,T1O_1_5,T1U_1_5,T2O_1_0,T2U_1_0,TO_1_75,TU_1_75,T2O_0_5,T2U_0_5,"AWAY_HDP -1,75","AWAY_HDP 1,25","HOME_HDP 3,5","AWAY_HDP 3,5","HOME_HDP 2,75","AWAY_HDP 2,75","HOME_HDP 2,5","AWAY_HDP 2,5","HOME_HDP 2,0","AWAY_HDP 2,0","HOME_HDP 1,75",TO_4_0,TU_4_0,TO_6_5,TU_6_5,TO_5_5,TU_5_5,TO_4_25,TU_4_25,TO_3_75,TU_3_75,T1O_1_0,T1U_1_0,T2O_3_5,T2U_3_5,T2O_1_5,T2U_1_5,"HOME_HDP -2,25",T2O_2_5,T2U_2_5,"HOME_HDP -3,5","AWAY_HDP -3,5",T1O_2_5,T1U_2_5,T2O_3_0,T2U_3_0,T1O_2_0,T1U_2_0,T1O_0_5,T1U_0_5,"HOME_HDP -2,75","AWAY_HDP -2,75",T1O_3_5,T1U_3_5,TO_1_25,TU_1_25,TO_0_5,TU_0_5,T2O_2_0,T2U_2_0,"HOME_HDP -3,0","AWAY_HDP -3,0",T1O_3_0,T1U_3_0,T2U_4_5,T2O_4_5,"HOME_HDP -3,25","AWAY_HDP 3,25","HOME_HDP 3,0","AWAY_HDP 3,0",TO_4_75,TU_4_75,TO_1_0,TU_1_0,TO_0_75,TU_0_75,TO_5_0,TU_5_0,TO_5_25,TU_5_25,T1U_4_5,T1O_4_5,TO_5_75,TU_5_75,TO_6_0,TU_6_0,TO_6_25,TU_6_25,QueueTime,InternalID,StartTime,BookmakerName,LineKindName
0,3.15,1.395,3.64,1.315,2.09,1.813,1.261,4.12,1.591,2.47,1.8,2.1,2.66,1.523,2.08,1.806,1.373,3.21,2.48,1.578,3.16,1.381,6.21,1.137,1.847,2.03,1.239,4.26,1.606,2.41,3.64,2.1,3.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch


In [24]:
zzz_for_check = book_odds.groupby(['InternalID', 'StartTime', 'BookmakerName']).agg(max)

In [ ]:
zzz_for_check.shape